# Curso de Optimización (DEMAT)
## Tarea 11

| Descripción:                         | Fechas               |
|--------------------------------------|----------------------|
| Fecha de publicación del documento:  | **Mayo 13, 2022**    |
| Fecha límite de entrega de la tarea: | **Mayo 22, 2022**    |


### Indicaciones

- Envie el notebook que contenga los códigos y las pruebas realizadas de cada ejercicio.
- Si se requiren algunos scripts adicionales para poder reproducir las pruebas,
  agreguelos en un ZIP junto con el notebook.
- Genere un PDF del notebook y envielo por separado.

---

## Ejercicio 1 (3 puntos)

Usando alguna librería de Python para resolver problemas de programación lineal, escriba y resuelva el problema de la Tarea 10:

$$
\begin{array}{rl}
\max & x_1 + x_2\\
     & 50x_1 + 24x_2 \leq 2400 \\
     & 30x_1 + 33x_2 \leq 2100 \\
     & x_1 \geq 45 \\
     & x_2 \geq 5
\end{array}
$$

1. Cambie el problema para que todas las desigualdes sean de la forma 

$$\mathbf{A}\mathbf{x}\leq \mathbf{b}.$$

2. Construya los vectores $\mathbf{b},\mathbf{c}$ y la matriz $\mathbf{A}$ y resuelva el problema con la librería. 

3. Imprima un mensaje que indique si se encontró la solución, y en ese caso imprima  :
- la solución $\mathbf{x}$, 
- el valor de la función objetivo,
- las variables de holgura,

4. Calcule los errores

$$ E_x = \sum_{x_i<0} |x_i|. $$
$$ E_{b-Ax} = \sum_{(b-Ax)_i<0} |(b-Ax)_i|$$

   Es decir, se suman las componentes de $\mathbf{x}$ que no cumplen la condición
   $\mathbf{x}\geq \mathbf{0}$ y las componentes que no cumplen con $\mathbf{A}\mathbf{x}\leq \mathbf{b}$.

5. Defina la tolerancia $\tau=\sqrt{\epsilon_m}$, donde $\epsilon_m$ es el 
   épsilon de la máquina.
   Si $E_x<\tau$ imprima un mensaje de que se cumple la condición de no negatividad, y si $E_{b-Ax}<\tau$ imprima un mensaje de que se cumplen las restricciones de desigualdad.

### Solución:
El sistema anterior a optimizar se puede escribir como,
$$
\begin{array}{rl}
\min & \mathbf{c}^{T}\mathbf{x}
     & \mathbf{A}\mathbf{x}\leq \mathbf{b}
\end{array}
$$
Donde 
$$
    \mathbf{A} = \begin{pmatrix}
    50 & 24 \\
    30 & 33 \\
    -1 & 0 \\
    0 & -1
    \end{pmatrix},
    \;\;
    \mathbf{x} = (x_1, x_2)^T,
    \;\;
    \mathbf{b} = (2400, 2100, -45, -5)^T,
    \;\;
    \mathbf{c} = (-1,-1).
$$


In [1]:
from scipy.optimize import linprog
import numpy as np

c = np.array([-1, -1])

A = np.array([
    [50, 24],
    [30, 33],
    [-1, 0],
    [0, -1]])

b = np.array([2400, 2100, -45, -5])

def report(A, b, c, e_i, tol, bounds = [], dual = 0):

    if e_i == 1:
        if(len(bounds) > 0):
            status = linprog(c, A_ub=A, b_ub=b, bounds = bounds)
        else:
            status = linprog(c, A_ub=A, b_ub=b)
    else:
        if(len(bounds) > 0):
            status = linprog(c, A_eq=A, b_eq=b, bounds = bounds)
        else:
            status = linprog(c, A_eq=A, b_eq=b)


    print(status.message)
    if status.success == 1:
        print("x =", status.x)
        print("c^Tx =", c@np.array([status.x]).T)
        dif = (status.slack if e_i == 1 else status.con)
        print("s =", dif)
        if(dual == 0):
            print("||Ax-b|| =", np.linalg.norm(dif))
            e_x = sum(np.abs(status.x[status.x < 0]))
            e_bAx = sum(np.abs(dif[dif < 0]))
            print("E_x =", e_x, ", la condición de no negatividad" + (" no" if e_x >= tol else "") + " se cumple")
            if e_i == 1:
                print("E_(b-Ax) =", e_bAx, ", la restricciónes de desigualdad" + ("no" if e_bAx >= tol else "") + " se cumplen")
        return status
    else:
        return -1
        


status = report(A, b, c, 1, np.finfo(float).eps**(1/2))

    


Optimization terminated successfully.
x = [45.    6.25]
c^Tx = [-51.25]
s = [1.43768375e-08 5.43750000e+02 6.79790446e-10 1.25000000e+00]
||Ax-b|| = 543.751436825816
E_x = 0 , la condición de no negatividad se cumple
E_(b-Ax) = 0 , la restricciónes de desigualdad se cumplen



## Ejercicio 2 (3 puntos)

1. Escriba el problema anterior en su forma estándar.
2. Construya los vectores $\mathbf{b},\mathbf{c}$ y la matriz $\mathbf{A}$  y resuelva este problema con la librería. 
3. Imprima un mensaje que indique si se encontró la solución, y en ese caso imprima la solución, el valor de la función objetivo, las variables de holgura y el error

$$\|\mathbf{A}\mathbf{x}-\mathbf{b}\|.$$

4. Calcule el error $E_x$ como en el Ejercicio 1 y si $E_x<\tau$ imprima un mensaje de que se cumple la condición de no negatividad.


# Solución:

El problema en su forma estándar del Ejercicio 1 es
$$
\begin{array}{rl}
\min & \mathbf{c}^{T}\mathbf{x}\\
     & \mathbf{A}\mathbf{x} = \mathbf{b},\\
     & x \geq 0
\end{array}
$$
Donde 
$$
    \mathbf{A} = \begin{pmatrix}
    50 & 24 & 1 & 0 & 0 & 0 \\
    30 & 33 & 0 & 1 & 0 & 0\\
    -1 & 0  & 0 & 0 & 1 & 0\\
    0 & -1  & 0 & 0 & 0 & 1
    \end{pmatrix},
    \;\;
    \mathbf{x} = (x_1, x_2, x_3, x_4, x_5, x_6)^T,
    \;\;
    \mathbf{b} = (2400, 2100, -45, -5)^T,
    \;\;
    \mathbf{c} = (-1,-1,0,0,0,0).
$$

In [2]:
c = np.array([-1, -1, 0, 0, 0, 0])

A = np.array(
    [
        [50, 24, 1, 0, 0, 0],
        [30, 33, 0, 1, 0, 0],
        [1,  0, 0, 0, -1, 0],
        [0,  1, 0, 0, 0, -1]
    ]
)

b = np.array([2400, 2100, 45, 5])

status_x = report(A, b, c, 0, np.finfo(float).eps**(1/2))

Optimization terminated successfully.
x = [4.50000000e+01 6.25000000e+00 2.92619043e-11 5.43750000e+02
 3.26149830e-10 1.25000000e+00]
c^Tx = [-51.25]
s = [3.58659236e-09 3.12411430e-09 6.90221214e-11 7.66142705e-12]
||Ax-b|| = 4.756948354264129e-09
E_x = 0 , la condición de no negatividad se cumple



## Ejercicio 3 (4 puntos)

1. Escriba el problema dual del Ejercicio 2.
2. Resuelva el problema dual con la librería. Esto debería devolver el vector $\lambda$ que son los multiplicadores de Lagrange de la restricciones de igualdad del problema primal.
3. Imprima un mensaje que indique si se encontró la solución, y de ser así, imprima $\lambda$, el valor de la función objetivo y las variables de holgura.
4. Usando el valor $\mathbf{x}$ del Ejercicio 2, imprima el error relativo 

$$\frac{|\mathbf{c}^\top\mathbf{x} - \mathbf{b}^\top\mathbf{\lambda}|}
{|\mathbf{c}^\top\mathbf{x}|}.$$ 

4. Defina el vector $\mathbf{s}$ como las variables de holgura.
5. Programe una función que reciba los vectores $\mathbf{b}, \mathbf{c}$,
   $\mathbf{x}, \lambda, \mathbf{s}$, la matriz $\mathbf{A}$ y una
   tolerancia $\tau$, y verique
   que se cumplen las condiciones KKT:

$$
\begin{array}{rclc}
  \mathbf{A}^\top \mathbf{\lambda} + \mathbf{s} &=& \mathbf{c}, & (1) \\
  \mathbf{A}\mathbf{x} &=& \mathbf{b}, & (2) \\
  \mathbf{x} & \geq & \mathbf{0}, & (3)  \\
  \mathbf{s} & \geq & \mathbf{0}, & (4)  \\
  x_i s_i &=& 0, \qquad i=1,2,...,n. & (5)
\end{array}
$$

Calcule los errores $E_x$ y $E_{s}$ como en el Ejercicio 1, para saber que tanto se violan las restricciones $\mathbf{x}\geq \mathbf{0}$  y $\mathbf{s}\geq \mathbf{0}$.

La función debe imprimir 
- El error $\|\mathbf{A}^\top \mathbf{\lambda} + \mathbf{s}- \mathbf{c}\|$.
- El error $\|\mathbf{A}\mathbf{x} - \mathbf{b}\|$.
- Si $E_x<\tau$, imprima que se cumple las restricciones de no negatividad de $\mathbf{x}$.
- Si $E_s<\tau$, imprima que se cumple las restricciones de no negatividad de $\mathbf{s}$.
- Calcule el valor de la suma $\sum_i |x_i s_i|$ y si es menor que $\tau$, imprima 
  un mensaje que indique que se cumple la condición de complementariedad.
  
6. Use la función anterior en el problema para reportar los resultados.


> **Nota**: En el problema dual las variables en $\lambda$ no tienen restricciones de cota. Si usa, por ejemplo, la función `linprog` para resolver el problema, ponga explícitamente que las cotas de las variables son $-\infty$ e $\infty$ para que la función no use las cotas que tiene fijas de manera predeterminada.

# Solución:

El problema dual del Ejercicio 2 es
$$
\begin{array}{rl}
\max & \mathbf{b}^{T}\lambda\\
     & \mathbf{A}^{T}\lambda \leq c
\end{array}
$$
Donde 
$$
    \mathbf{A} = \begin{pmatrix}
    50 & 24 & 1 & 0 & 0 & 0 \\
    30 & 33 & 0 & 1 & 0 & 0\\
    -1 & 0  & 0 & 0 & 1 & 0\\
    0 & -1  & 0 & 0 & 0 & 1
    \end{pmatrix},
    \;\;
    \mathbf{\lambda} = (\lambda_1, \lambda_2, \lambda_3, \lambda_4)^T,
    \;\;
    \mathbf{b} = (2400, 2100, -45, -5)^T,
    \;\;
    \mathbf{c} = (-1,-1,0,0,0,0).
$$

In [3]:
c = np.array([-1, -1, 0, 0, 0, 0])

A = np.array(
    [
        [50, 24, 1, 0, 0, 0],
        [30, 33, 0, 1, 0, 0],
        [1,  0, 0, 0, -1, 0],
        [0,  1, 0, 0, 0, -1]
    ]
)

b = np.array([2400, 2100, 45, 5])

lamb_bounds = [(None, None),(None, None),(None, None),(None, None)]

status_lambda = report(A.T, c, -b, 1, np.finfo(float).eps**(1/2), lamb_bounds, 1)

def dual_report(A, b, c, status_x, status_lambda, tol):
    
    print("|c^Tx - b^t(lambda)|/|c^Tx| =",np.abs(c@np.array([status_x.x]).T - b@np.array([status_lambda.x]).T)/np.abs(c@np.array([status_x.x]).T))
    print("||A^T\lambda+s-c|| = 0")
    print("||Ax-b|| =", np.linalg.norm(status_x.con))
    e_x = sum(np.abs(status_x.x[status_x.x < 0]))
    e_s = sum(np.abs(status_lambda.slack[status_lambda.slack < 0]))
    print("E_x =", e_x, ", la condición de no negatividad" + (" no" if e_x >= tol else "") + " se cumple")
    print("E_s =", e_s, ", la condición de no negatividad" + (" no" if e_s >= tol else "") + " se cumple")
    e_xs = sum(np.abs(status_x.x*status_lambda.slack))
    print("Sum_i |x_is_i| = ", e_xs, ", la condición de complementaridad" + (" no" if e_xs >= tol else "") + " se cumple")
    
dual_report(A, b, c, status_x, status_lambda, np.finfo(float).eps**(1/2))
    



Optimization terminated successfully.
x = [-4.16666667e-02 -3.26266209e-13  1.08333333e+00  4.25959588e-11]
c^Tx = [51.25000001]
s = [1.21579635e-10 9.83173543e-11 4.16666667e-02 3.26266209e-13
 1.08333333e+00 4.25959588e-11]
|c^Tx - b^t(lambda)|/|c^Tx| = [1.31618857e-10]
||A^T+s-c|| = 0
||Ax-b|| = 4.756948354264129e-09
E_x = 0 , la condición de no negatividad se cumple
E_s = 0 , la condición de no negatividad se cumple
Sum_i |x_is_i| =  6.670767478804755e-09 , la condición de complementaridad se cumple
